<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/Correct_create_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/xechehot/sirius-2021-warehouse

fatal: destination path 'sirius-2021-warehouse' already exists and is not an empty directory.


In [3]:
! cd sirius-2021-warehouse; git pull

Already up to date.


In [9]:
from json import load as JLoad

with open('sirius-2021-warehouse/data_sample/sample_64_1.json', 'r') as f:
  data = JLoad(f) 

#data

In [10]:
floors = data['warehouse']['meta']['floors']
block_x = data['warehouse']['meta']['block_x']
block_y = data['warehouse']['meta']['block_y']
rows = data['warehouse']['meta']['rows'] 
sections = data['warehouse']['meta']['sections']

floors, block_x, block_y, rows, sections

(1, 1, 2, 4, 8)

In [17]:
import json

Map_y = (3 * rows) // 2 + 1
Map_x = sections + 2
 
s_map_x = block_x * Map_x
s_map_y = block_y * Map_y

map_for_block = [[0 for i in range(s_map_y)] for j in range(s_map_x)]


MAP = []

for i in range(Map_x):
  MAP.append([0 for _ in range(Map_y)])
  if i == 0 or i == Map_x - 1:
    continue
  for j in range(Map_y):
    if j % 3 != 0:
      MAP[i][j] = 1


# print(Map_x, Map_y)



# for i in MAP:
#  for j in i:
  #  print(j, end = ' ')
#  print('')


for i in range(s_map_x):
  for j in range(s_map_y):
    map_for_block[i][j] = MAP[i % Map_x][j % Map_y]

for i in range (s_map_y):
  map_for_block[0][i] = 2

startx = s_map_x - 1
starty = s_map_y // 2

# for i in map_for_block:
#  for j in i:
  #  print(j, end = ' ')
#  print('')

output = {
      "map": {
          "dimensions": {
              "x": s_map_x,
              "y": s_map_y
          },
          "shelves": [
                      
          ],
          "dropZone": [
                  
          ],
          "start": {
            "x": s_map_x - 1,
            "y": s_map_y // 2 
        }
      },
      "items": [
                
      ],
}

shelves = []
dropZone = []
for i in range(s_map_x):
    for j in range(s_map_y):
        if map_for_block[i][j] == 1:
            item = {
                "x": i,
                "y": j
            }
            shelves.append(item)
        elif map_for_block[i][j] == 2:
            item = {
                "x": i,
                "y": j
            }
            dropZone.append(item)
output['map']['shelves'] = shelves
output['map']['dropZone'] = dropZone
with open("visual_sample_map.json", "w") as f:
  json.dump(output, f)
# print(json.dumps(output, indent=3))
